In [ ]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### load data

In [ ]:
adata_tabula = sc.read_h5ad('./io/tabula_sapiens/rawdata/unzipped_files/TabulaSapiens.h5ad')

In [ ]:
adata_tabula

### select highly variable genes

In [ ]:
adata_tabula_highly = adata_tabula[:, adata_tabula.var.highly_variable]

In [ ]:
adata_tabula_highly

In [ ]:
adata_tabula_highly.obs.cell_ontology_class.value_counts()

In [ ]:
adata_tabula_highly_arr = adata_tabula_highly.X.toarray()

In [ ]:
adata_tabula_highly_Y = adata_tabula_highly.obs.cell_ontology_class

In [ ]:
adata_tabula_highly_Y

In [ ]:
np.save('./io/tabular_sapiens/output/adata_tabula_highly_X.npy', adata_tabula_highly_arr)

In [ ]:
adata_tabula_highly_Y.to_csv('./io/tabular_sapiens/output/adata_tabula_Y.tsv', header=False
                             , index=False
                             , sep='\t')

In [ ]:
adata_tabula_highly_genes = adata_tabula_highly.var.gene_symbol

In [ ]:
adata_tabula_highly_genes.to_csv('./io/tabular_sapiens/output/adata_tabula_highly_genes.tsv', header=False
                             , index=False
                             , sep='\t')

### filter genes

In [ ]:
adata_tabula_filter_gene = sc.pp.filter_genes(adata_tabula, min_cells=int(483152 * 0.001), inplace=False)

In [ ]:
adata_tabula_filter_gene = adata_tabula[:, adata_tabula_filter_gene[0]]

In [ ]:
adata_tabula_subset_X = adata_tabula_filter_gene.X.toarray()

In [ ]:
adata_tabula_subset_X.shape

In [ ]:
adata_tabula_subset_Y = adata_tabula_filter_gene.obs.cell_ontology_class

In [ ]:
np.save('./io/tabular_sapiens/output/filter_gene_483/X.npy', adata_tabula_subset_X)

In [ ]:
adata_tabula_subset_Y.to_csv('./io/tabular_sapiens/output/filter_gene_483/Y.tsv', header=False
                             , index=False
                             , sep='\t')

In [ ]:
adata_tabula_subset_genes = adata_tabula_filter_gene.var.gene_symbol

In [ ]:
adata_tabula_subset_genes.shape

### split train and evaluation, save to drive

In [ ]:
import datasets
from datasets import Dataset
import math

In [ ]:
data_dir = './io/tabular_sapiens/output/filter_gene_483'
Y = list(pd.read_csv(os.path.join(data_dir, 'Y.tsv'), sep='\t', header=None).iloc[:,0])

In [ ]:
X = np.load(os.path.join(data_dir, 'X.npy'))

In [ ]:
num_samples = X.shape[0]
max_single_ds = 10000

In [ ]:
X.shape

In [ ]:
Y = np.array(Y)

In [ ]:
Y.shape

In [ ]:
label_dict, label = np.unique(Y, return_inverse=True) 

In [ ]:
label.shape

In [ ]:
label_dict.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=7747)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
np.unique(y_train).shape

In [ ]:
np.unique(y_test).shape

In [ ]:
np.save('./io/tabular_sapiens/output/filter_gene_483/X_train.npy', X_train)
np.save('./io/tabular_sapiens/output/filter_gene_483/y_train.npy', y_train)
np.save('./io/tabular_sapiens/output/filter_gene_483/X_test.npy', X_test)
np.save('./io/tabular_sapiens/output/filter_gene_483/y_test.npy', y_test)

np.save('./io/tabular_sapiens/output/filter_gene_483/labels.npy', label_dict)

In [ ]:
output_dir = '/home/zli17/work/projects/scBERT/io/scbert/small_for_test'
np.save(f'{output_dir}/X_train.npy', X_train[:200])
np.save(f'{output_dir}/y_train.npy', y_train[:200])
np.save(f'{output_dir}/X_test.npy', X_test[:100])
np.save(f'{output_dir}/y_test.npy', y_test[:100])

np.save(f'{output_dir}/label_dict.npy', label_dict)

In [ ]:
label2id = {s:i for i,s in enumerate(label_dict)}

### group genes by pathway

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [ ]:
"""
this json file is downloaded on April 14, 2022 using this link: 
[https://www.genome.jp/kegg-bin/get_htext?hsa00001+3101
"""
with open('../io/kegg/hsa00001.json', 'r') as f:
    kegg_json = json.load(f)

In [ ]:
pathways = {}
for l1 in kegg_json['children']:
    print(l1['name'])
    for l2 in l1['children']:
        print('\t', l2['name'])
        for l3 in l2['children']:
            l3_children = l3.get('children')
            if l3_children:
                print('\t\t', len(l3_children), l3['name'])
            else:
                print('\t\t', 0, l3['name'])

In [ ]:
res_l = []
for l1 in kegg_json['children']:
    l1_name = l1.get('name')
    l1_id = l1_name.split(' ')[0]
    for l2 in l1['children']:
        l2_name = l2.get('name')
        l2_id = l2_name.split(' ')[0]
        for l3 in l2['children']:
            l3_children = l3.get('children')
            l3_name = l3.get('name')
            l3_id = l3_name.split(' ')[0]
            if l3_children:
                for l4 in l3_children:
                    splits = l4['name'].split(';')[0].split(' ')
                    geneid = splits[0]
                    genesymbol = splits[1]
                    res_l.append([geneid, genesymbol, l3_id, l3_name, l2_id, l2_name, l1_id, l1_name])
kegg_df = pd.DataFrame(res_l, columns=['GeneID', 'Symbol', 'L3ID', 'L3Name', 'L2ID', 'L2Name', 'L1ID', 'L1Name'])

In [ ]:
kegg_df.head()

In [ ]:
kegg_overlap_df = kegg_df[kegg_df.Symbol.isin(adata_tabula_subset_genes)].loc[:, ['Symbol', 'L3ID']]
kegg_overlap_df.shape

In [ ]:
kegg_overlap_df.groupby('L3ID').count().describe()

In [ ]:
overlapped_genes = kegg_overlap_df.Symbol.unique()

In [ ]:
i = 0
random_group_size = 110
left_genes = []
for gene in adata_tabula_subset_genes:
    if gene not in overlapped_genes:
        left_genes.append([gene, f'NO_PATHWAY_{int(i/random_group_size)}'])
        i += 1
print(f'Left Genes: {len(left_genes)}')

In [ ]:
left_gene_df = pd.DataFrame(left_genes, columns=['Symbol', 'L3ID'])

In [ ]:
gene_pathway_df = pd.concat([kegg_overlap_df, left_gene_df])
gene_pathway_df.shape

In [ ]:
gene_pathway_df.L3ID.value_counts()

In [ ]:
pathway_onehot = pd.get_dummies(gene_pathway_df, columns=['L3ID'], prefix='Pathway')
pathway_onehot.shape

In [ ]:
gene_group = pathway_onehot.groupby('Symbol').sum()

In [ ]:
gene_group = gene_group.reset_index()

In [ ]:
gene_group.head()

In [ ]:
gene_group_reorder = pd.merge(adata_tabula_subset_genes, gene_group, how='left', left_on='gene_symbol', right_on='Symbol')

In [ ]:
gene_group_reorder.head()

In [ ]:
gene_group_reorder.tail()

In [ ]:
adata_tabula_subset_genes

In [ ]:
gene_group_reorder = gene_group_reorder.set_index('Symbol')

In [ ]:
gene_group_reorder.head()

In [ ]:
gene_group_reorder = gene_group_reorder.iloc[:, 1:]

In [ ]:
gene_group_reorder.head()

In [ ]:
gene_group_reorder.shape

In [ ]:
gene_group_reorder.to_csv('./io/tabular_sapiens/output/filter_gene_483/kegg_grouping.tsv', index=True, header=True, sep='\t')

### random grouping

In [ ]:
gene_group_reorder = pd.read_csv('./io/tabular_sapiens/output/filter_gene_483/kegg_grouping.tsv', index_col='Symbol'
                                 ,  sep='\t')

In [ ]:
gene_group_reorder.shape

In [ ]:
gene_group_reorder.head()

In [ ]:
gene_random_group_df = pd.DataFrame({'Symbol': gene_group_reorder.index,
                                    'Group': list(np.random.randint(low=0, high=gene_group_reorder.shape[1], size=(gene_group_reorder.shape[0])))}) 

In [ ]:
gene_random_group_df.head()

In [ ]:
gene_random_group_df.shape

In [ ]:
gene_random_group_onehot = pd.get_dummies(gene_random_group_df, columns=['Group'], prefix='Group')
gene_random_group_onehot.shape

In [ ]:
gene_random_group_onehot.head()

In [ ]:
gene_random_group_onehot = gene_random_group_onehot.set_index('Symbol')

In [ ]:
gene_random_group_onehot.shape

In [ ]:
gene_random_group_onehot.to_csv('./io/tabular_sapiens/output/filter_gene_483/random_grouping.tsv', index=True, header=True, sep='\t')

### convert to dataset and save

In [ ]:
import datasets
from datasets import Dataset
import os
import numpy as np
import math

In [ ]:
def load_dataset(data_dir):
    X_train = np.load(os.path.join(data_dir, f'X_train.npy'))
    X_eval = np.load(os.path.join(data_dir, f'X_test.npy'))
    Y_train = np.load(os.path.join(data_dir, f'y_train.npy'))
    Y_eval = np.load(os.path.join(data_dir, f'y_test.npy'))

    ds = {}
    for k,X in {'train':X_train, 'test': X_eval}.items():
        if k == 'train':
            Y = Y_train
        else:
            Y = Y_eval
        num_samples = X.shape[0]
        max_single_ds = 10000

        if num_samples <= max_single_ds:
            dataset = Dataset.from_dict({'gex': X, 'label': Y})
        else:
            ds_l = []
            for i in range(math.ceil(num_samples/max_single_ds)):
                dataset_i = datasets.Dataset.from_dict(
                    {'gex': X[i * max_single_ds:(i + 1) * max_single_ds, :], 'label': Y[i * max_single_ds:(i + 1) * max_single_ds]})
                ds_l.append(dataset_i)
            dataset = datasets.concatenate_datasets(ds_l)
        ds[k] = dataset
    return ds

In [ ]:
output_dir = './io/tabular_sapiens/output/filter_gene_483'
ds = load_dataset(output_dir)

In [ ]:
ds_output_dir = './io/tabular_sapiens/output/filter_gene_483_ds_cache'
if not os.path.exists(ds_output_dir):
    os.makedirs(ds_output_dir)
for k,v in ds.items():
    v.save_to_disk(os.path.join(ds_output_dir, f'{k}'))

In [ ]:
import shutil
output_dir = './io/tabular_sapiens/output/filter_gene_483'
shutil.copyfile(os.path.join(output_dir, 'labels.npy'), os.path.join(ds_output_dir, 'labels.npy'))